In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import numpy as np

import metrics

round_digits = 4

# Define the folder and file filters

image_data_folder = "../results/image_classification/epochs/"
image_filters = [
    {"name": "ResNet20", "file_name_filter": "ResNet"},
    {"name": "ViT-S-8", "file_name_filter": "ViT"}
]
image_results_column = "test_accuracy"

time_data_folder = "../results/time_series/epochs/"
time_filters = [
    {"name": "Autoformer", "file_name_filter": "Autoformer"},
    {"name": "iTransformer", "file_name_filter": "iTransformer"},
    {"name": "NLinear", "file_name_filter": "NLinear"},
    {"name": "TSMixer", "file_name_filter": "TSMixer"},
]
time_results_column = "mae"

In [ ]:
def load_results(filters, data_folder, distribution_column, multiplier=1):
    # Initialize a dictionary to store results for each filter
    results = {}

    # Process each filter
    for filter_item in filters:
        file_name_filter = filter_item["file_name_filter"]
        name = filter_item["name"]
        
        # Get all CSV files in the data folder that start with file_name_filter
        file_names = sorted(
            [f for f in os.listdir(data_folder) if f.startswith(file_name_filter) and f.endswith(".csv")]
        )
        
        # Dictionary to store the epoch and corresponding statistics of test_accuracy
        epoch_values = {}
        
        # Process each file
        for file_name in file_names:
            file_path = os.path.join(data_folder, file_name)
            df = pd.read_csv(file_path)
            
            # Extract the number of epochs from the filename
            epoch_match = re.search(r'e(\d+)', file_name)
            if epoch_match:
                epoch = int(epoch_match.group(1))
                # Get the test_accuracy column and calculate the mean and range
                distribution = df[distribution_column].to_list()
                distribution = [x * multiplier for x in distribution]
                mean_value = metrics.mean(distribution)
                range_value = metrics.range(distribution)
                epoch_values[epoch] = {"mean": mean_value, "range": range_value}
        
        # Sort the results by epoch
        sorted_epochs = sorted(epoch_values.keys())
        mean_values = [epoch_values[ep]["mean"] for ep in sorted_epochs]
        range_values = [epoch_values[ep]["range"] for ep in sorted_epochs]
        
        # Store the results
        results[name] = {"epochs": sorted_epochs, "means": mean_values, "ranges": range_values}

    return results

In [ ]:
def results_to_dataframe(results, round_digits):
    # Create a dataframe summarizing the results
    summary_data = []
    for model_name, model_data in results.items():
        for epoch, mean, range_ in zip(model_data['epochs'], model_data['means'], model_data['ranges']):
            summary_data.append({
                'Model': model_name,
                'Epoch': epoch,
                'Mean': metrics.safe_round(mean, round_digits),
                'Range': metrics.safe_round(range_, round_digits)
            })

    summary_df = pd.DataFrame(summary_data)
    return summary_df

In [ ]:
image_results = load_results(image_filters, image_data_folder, image_results_column, multiplier=100)
time_results = load_results(time_filters, time_data_folder, time_results_column)

In [ ]:
image_df = results_to_dataframe(image_results, 2)

# Split into separate DataFrames
resnet_df = image_df[image_df['Model'] == 'ResNet20'].reset_index(drop=True)
vit_df = image_df[image_df['Model'] == 'ViT-S-8'].reset_index(drop=True)

# Print or display the tables
print("ResNet Evaluation:\n", resnet_df)
corr_r, corr_p = metrics.correlation(resnet_df["Mean"].to_list(), resnet_df["Range"].to_list())
print("Correlation (ResNet): r =", metrics.safe_round(corr_r, round_digits))

print("\nViT Evaluation:\n", vit_df)
corr_r, corr_p = metrics.correlation(vit_df["Mean"].to_list(), vit_df["Range"].to_list())
print("Correlation (ViT): r =", metrics.safe_round(corr_r, round_digits))


In [ ]:
time_df = results_to_dataframe(time_results, 4)

# Split into separate DataFrames
autoformer_df = time_df[time_df['Model'] == 'Autoformer'].reset_index(drop=True)
itransformer_df = time_df[time_df['Model'] == 'iTransformer'].reset_index(drop=True)
nlinear_df = time_df[time_df['Model'] == 'NLinear'].reset_index(drop=True)
tmsixer_df = time_df[time_df['Model'] == 'TSMixer'].reset_index(drop=True)

# Print or display the tables
print("Autoformer Evaluation:\n", autoformer_df)
corr_r, corr_p = metrics.correlation(autoformer_df["Mean"].to_list(), autoformer_df["Range"].to_list())
print("Correlation (Autoformer): r =", metrics.safe_round(corr_r, round_digits))

print("\niTransformer Evaluation:\n", itransformer_df)
corr_r, corr_p = metrics.correlation(itransformer_df["Mean"].to_list(), itransformer_df["Range"].to_list())
print("Correlation (iTransformer): r =", metrics.safe_round(corr_r, round_digits))

print("\nNLinear Evaluation:\n", nlinear_df)
corr_r, corr_p = metrics.correlation(nlinear_df["Mean"].to_list(), nlinear_df["Range"].to_list())
print("Correlation (NLinear): r =", metrics.safe_round(corr_r, round_digits))

print("\nTSMixer Evaluation:\n", tmsixer_df)
corr_r, corr_p = metrics.correlation(tmsixer_df["Mean"].to_list(), tmsixer_df["Range"].to_list())
print("Correlation (TSMixer): r =", metrics.safe_round(corr_r, round_digits))

In [ ]:
# Plot the results
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot mean accuracy on the left Y-axis
for name, data in image_results.items():
    ax1.plot(data["epochs"], data["means"], marker='o', label=f"{name} Mean")
ax1.set_xlabel("Number of Epochs", fontsize=15)
ax1.set_ylabel("Mean Top-1 Accuracy (%)", fontsize=15, color="blue")
ax1.tick_params(axis="y", labelsize=12, labelcolor="blue")
ax1.tick_params(axis="x", labelsize=12)

# Create a second Y-axis for the range of accuracy
ax2 = ax1.twinx()
for name, data in image_results.items():
    ax2.plot(data["epochs"], data["ranges"], marker='x', linestyle='--', label=f"{name} Range")
ax2.set_ylabel("Range Top-1 Accuracy (%)", fontsize=15, color="red")
ax2.tick_params(axis="y", labelsize=12, labelcolor="red")

# Add title and legend
plt.title("Mean and Range of Accuracy vs Number of Epochs\n(ResNet20 and ViT-S-8 with CIFAR-10)", fontsize=16)
fig.legend(loc="upper center", fontsize=15, bbox_to_anchor=(.5, -.01), ncol=4)
plt.tight_layout()

# Save the plot
plot_file_path = "plots/rq3/image.pdf"
plt.savefig(plot_file_path, dpi=600, bbox_inches='tight')  # Save the plot with proper bounding

plt.show()  # Show the plot
plt.close()  # Close the plot to ensure it doesn't interfere with subsequent plots

In [ ]:
# Group models
left_models = ["Autoformer"]
right_models = ["iTransformer", "NLinear", "TSMixer"]
epoch_indices = list(range(1, 11))

# Create figure and axes
#fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
fig, (ax_left, ax_right) = plt.subplots(2, 1, figsize=(10, 12), sharey=True)


# ================================
# LEFT PLOT: Autoformer
# ================================
color_autoformer = 'tab:blue'
for name in left_models:
    data = time_results[name]
    ax_left.plot(epoch_indices, data["means"], marker='o', linestyle='-', color=color_autoformer, label=f"{name} Mean", zorder=2)
ax_left.set_xlabel("Number of Epochs", fontsize=13)
ax_left.set_ylabel("Mean of MAE", fontsize=15, color="blue")
ax_left.tick_params(axis="y", labelsize=12, labelcolor="blue")
ax_left.tick_params(axis="x", labelsize=12)

# Range (right Y-axis)
ax2_left = ax_left.twinx()
for name in left_models:
    data = time_results[name]
    ax2_left.plot(epoch_indices, data["ranges"], marker='x', linestyle='--', color=color_autoformer, label=f"{name} Range", zorder=1)
ax2_left.set_ylabel("Range of MAE", fontsize=15, color="red")
ax2_left.tick_params(axis="y", labelsize=12, labelcolor="red")

# Legend for left
lines_left, labels_left = ax_left.get_legend_handles_labels()
lines_left2, labels_left2 = ax2_left.get_legend_handles_labels()
ax_left.legend(lines_left + lines_left2, labels_left + labels_left2, loc="lower right", fontsize=14)
ax_left.set_title("Autoformer", fontsize=16)

# ================================
# RIGHT PLOT: iTransformer, NLinear, TSMixer
# ================================
colors = {
    "Autoformer": "tab:blue",
    "iTransformer": "tab:green",
    "NLinear": "tab:orange",
    "TSMixer": "tab:purple"
}

# Plot means
for name in right_models:
    data = time_results[name]
    ax_right.plot(epoch_indices, data["means"], marker='o', linestyle='-', color=colors[name], label=f"{name} Mean", zorder=2)

# Plot ranges (second Y-axis)
ax2_right = ax_right.twinx()
for name in right_models:
    data = time_results[name]
    ax2_right.plot(epoch_indices, data["ranges"], marker='x', linestyle='--', color=colors[name], label=f"{name} Range", zorder=1)

# Axis labels
ax_right.set_xlabel("Number of Epochs (iTransformer, NLinear)", fontsize=13)
ax_right.set_ylabel("Mean of MAE", fontsize=15, color="blue")
ax_right.tick_params(axis="y", labelsize=12, labelcolor="blue", labelleft=True)
ax_right.tick_params(axis="y", labelsize=12, labelcolor="blue")
ax2_right.set_ylabel("Range of MAE", fontsize=15, color="red")
ax2_right.tick_params(axis="y", labelsize=12, labelcolor="red")
ax_right.tick_params(axis="x", labelsize=12)
#ax_right.grid(True)

# Top x-axis for TSMixer
ax_top_right = ax_right.twiny()
ax_top_right.set_xlim(ax_right.get_xlim())
ax_top_right.tick_params(axis="x", labelsize=12)
ax_top_right.set_xticks(epoch_indices)
ax_top_right.set_xticklabels(time_results["TSMixer"]["epochs"])
ax_top_right.set_xlabel("Number of Epochs (TSMixer)", fontsize=13)

# Legend for right (top right corner)
lines_right, labels_right = ax_right.get_legend_handles_labels()
lines_right2, labels_right2 = ax2_right.get_legend_handles_labels()
ax_right.legend(lines_right + lines_right2, labels_right + labels_right2, loc="upper right", fontsize=14)
ax_right.set_title("iTransformer, NLinear, TSMixer", fontsize=16)

# ================================
# Final layout
# ================================
fig.suptitle("Mean and Range of MAE vs Number of Epochs\n(Autoformer, iTransformer, NLinear, & TSMixer Horizon 96 with ETTh1)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])

# Save and show
plot_file_path = "plots/rq3/time_series.pdf"
plt.savefig(plot_file_path, dpi=600, bbox_inches='tight')
plt.show()
plt.close()